<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/LLAMA3_SPACE_FP_OODA_LOOP_AI_AGENT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Pytorch & other libraries
!pip install torch tensorboard --quiet

# Install Hugging Face libraries
!pip install  --upgrade transformers datasets accelerate evaluate bitsandbytes --quiet

#FlashAttention only supports Ampere GPUs or newer. #NEED A100 OR L4 IN GOOGLE COLAB
#!pip install -U transformers
!pip install -U flash-attn --no-build-isolation --quiet


! pip install peft --quiet
! pip install datasets trl ninja packaging --quiet

# Uncomment only if you're using A100 GPU
#!pip install flash-attn --no-build-isolation
!pip install diffusers safetensors  --quiet
!pip install colab-env --quiet

In [2]:
import torch
import os
import sys
import json
import IPython
from datetime import datetime
from datasets import load_dataset
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
    pipeline,
)

In [3]:
import os
from typing import List, Dict
import logging
import pandas as pd
from sklearn.linear_model import LogisticRegression
import re
import time
from tqdm import tqdm

# Disable warnings from transformers
logging.getLogger("transformers.modeling_utils").setLevel(logging.ERROR)

import torch
from datetime import datetime

import warnings
warnings.filterwarnings("ignore")


# Specify CUDA device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Initialize Llama model and tokenizer
llama_model_id = "meta-llama/Llama-3.1-8B-Instruct"

# Initialize Llama 3 with padding token fix
llama_tokenizer = AutoTokenizer.from_pretrained(llama_model_id, use_fast=True)
llama_tokenizer.padding_side = "right"
llama_tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Initialize Llama 3
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
llama_model = AutoModelForCausalLM.from_pretrained(
    llama_model_id,
    device_map=device,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config
)

# Resize token embeddings with mean_resizing=False
llama_model.resize_token_embeddings(len(llama_tokenizer), mean_resizing=False)

# Move model to the device
llama_model.to(device)

llama_pipeline = pipeline(
    "text-generation",
    model=llama_model,
    tokenizer=llama_tokenizer,
    pad_token_id=llama_tokenizer.pad_token_id
)

llama_pipeline.generation_params = {
    "max_new_tokens": 2048, # 4096
    "do_sample": True,
    "top_k": 50,
    "top_p": 0.95,
    "temperature": 0.7,
    "pad_token_id": llama_tokenizer.pad_token_id,
}


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/gdrive')
daily_solar_data_space_df = pd.read_csv('/content/gdrive/MyDrive/datasets/SW/daily_solar_data.csv')

In [5]:
daily_solar_data_space_df

,Date,Radio Flux 10.7cm,Sunspot Number,Sunspot Area (10^6 Hemis.),New Regions,Stanford Mean Solar Field (GOES15),Stanford Background X-Ray Flux,Flares: C,Flares: M,Flares: X,Flares: S,Flares: 1,Flares: 2,Flares: 3
0,1997-01-01,72,0,0,0,*,A0.5,0,0,0,-1,-1,-1,-1
1,1997-01-02,72,0,0,0,*,A0.5,0,0,0,-1,-1,-1,-1
2,1997-01-03,73,0,0,0,3,A0.5,0,0,0,-1,-1,-1,-1
3,1997-01-04,74,13,10,1,*,A0.6,0,0,0,-1,-1,-1,-1
4,1997-01-05,74,15,20,0,5,A0.7,0,0,0,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10325,2024-04-08,125,79,470,1,-999,*,2,0,0,0,0,0,0
10326,2024-04-09,124,64,405,0,-999,*,2,0,0,0,0,0,0
10327,2024-04-10,131,54,410,1,-999,*,11,0,0,3,0,0,0
10328,2024-04-11,144,81,560,1,-999,*,10,1,0,4,0,0,0


In [6]:
space_weather_df = pd.read_csv('/content/gdrive/MyDrive/datasets/SW/daily_solar_data.csv')
space_weather_df['Date'] = pd.to_datetime(space_weather_df['Date'])

In [8]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import os
import re
import time
from tqdm import tqdm

space_weather="/content/gdrive/MyDrive/datasets/SW/daily_solar_data.csv"

# Load space weather data from the provided path
space_weather_df = pd.read_csv(space_weather)
space_weather_df["Date"] = pd.to_datetime(space_weather_df["Date"])


class SpaceEnvironment:
    def __init__(self, origin, destination, space_weather_df):
        self.origin = origin
        self.destination = destination
        self.space_weather_df = space_weather_df  # Store the DataFrame

    def get_possible_trajectories(self):
        # Define trajectories, including Lunar Gateway as a waypoint if needed
        trajectories = {
            ("Earth orbit", "Moon surface"): [
                {
                    "trajectory_id": "T1",
                    "delta_v": 3.9,
                    "duration": 3.5,
                    "fuel_required": 1200,
                    "waypoints": ["Lunar Gateway"],  # Include Lunar Gateway as a waypoint
                },
                # ... (add more trajectories) ...
            ]
        }
        return trajectories.get((self.origin, self.destination), [])

    def get_space_weather_data(self, location):
        # Extract data for the given location from the DataFrame
        latest_data = self.space_weather_df.iloc[-1]  # Get the latest data row
        # Adjust logic based on your data format and column names
        return {
            "solar_flare_risk": "High" if latest_data["Flares: C"] > 0 else "Low",  # Corrected column name
            "radiation_level": "Moderate",  # Replace with relevant logic based on other columns
        }


class SpaceFlightAgent:
    def __init__(self, environment):
        self.environment = environment
        self.space_weather_model = LogisticRegression()
        self.train_space_weather_model()

    def train_space_weather_model(self):
        # Feature engineering and model training
        space_weather_df["high_solar_activity"] = (
            (space_weather_df["Sunspot Number"] > 50)  # Corrected column name
            | (space_weather_df["Radio Flux 10.7cm"] > 150)  # Corrected column name
        ).astype(int)
        target = space_weather_df["high_solar_activity"]
        features = space_weather_df[["Sunspot Number", "Radio Flux 10.7cm"]]  # Corrected column names
        self.space_weather_model.fit(features, target)
        print("Space weather model trained successfully!")

    def generate_space_flight_plan(self):
        origin_weather = self.environment.get_space_weather_data(
            self.environment.origin
        )
        destination_weather = self.environment.get_space_weather_data(
            self.environment.destination
        )
        possible_trajectories = self.environment.get_possible_trajectories()

        print(f"Space weather at {self.environment.origin}: {origin_weather}")
        print(
            f"Space weather at {self.environment.destination}: {destination_weather}"
        )
        print("Available trajectories:", possible_trajectories)

        if not possible_trajectories:
            return "No trajectories found."

        # OODA Loop:
        print('\n')
        num_iterations = 5  # Define the number of iterations
        for _ in tqdm(range(num_iterations), desc="OODA Loop"):
            # Observe:
            # (In this example, we are using the existing space_weather_df)
            # For real-time updates, you would need to fetch new data here
            # and update the space_weather_df accordingly.

            # Orient:
            latest_data = self.environment.space_weather_df.iloc[-1]  # Latest data row
            current_sunspot_number = latest_data["Sunspot Number"]  # Corrected column name
            current_radio_flux = latest_data["Radio Flux 10.7cm"]  # Corrected column name

            origin_features = [[current_sunspot_number, current_radio_flux]]
            destination_features = [[current_sunspot_number, current_radio_flux]]

            # Create DataFrames with feature names for prediction
            origin_features_df = pd.DataFrame(origin_features, columns=["Sunspot Number", "Radio Flux 10.7cm"])
            destination_features_df = pd.DataFrame(destination_features, columns=["Sunspot Number", "Radio Flux 10.7cm"])

            origin_weather["solar_flare_risk"] = (
                "High"
                if self.space_weather_model.predict(origin_features_df)[0] == 1
                else "Low"
            )
            destination_weather["solar_flare_risk"] = (
                "High"
                if self.space_weather_model.predict(destination_features_df)[0] == 1
                else "Low"
            )

            # Decide:
            # ... (Add your decision logic here based on origin_weather,
            # destination_weather, etc.) ...
            chosen_trajectory = possible_trajectories[0]  # Example: Choose the first trajectory

            # Act:
            # Call Llama 3 to generate/update the plan
            prompt = f"""
            <s>[INST] Generate a detailed space flight plan for a mission from {self.environment.origin} to the {self.environment.destination},
            using NASA's Lunar Gateway as a staging point. Consider the following:
            - Chosen Trajectory: {chosen_trajectory}
            - Solar flare risk at origin: {origin_weather['solar_flare_risk']}
            - Solar flare risk at destination: {destination_weather['solar_flare_risk']}
            # ... (Add other instructions/details to the prompt) ...
            [/INST]"""

            response = llama_pipeline(prompt, max_new_tokens=2048)
            space_flight_plan = response[0]['generated_text']

            # Post-processing (may need adjustments)
            space_flight_plan = re.sub(r"\n+", "\n", space_flight_plan)
            space_flight_plan = re.sub(r"## (.*)", r"\n\n**\1**:", space_flight_plan)

            print('\n')
            print("\nGenerated Space Flight Plan:")
            print(space_flight_plan)
            print('\n')

                # ... (Add a delay to control loop frequency) ...
                # time.sleep(60)  # Example: Wait 60 seconds (commented out or adjusted)


# Example usage
environment = SpaceEnvironment(
    origin="Earth orbit",
    destination="Moon surface",
    space_weather_df=space_weather_df,  # Pass the DataFrame
)

agent = SpaceFlightAgent(environment)
agent.generate_space_flight_plan()

Space weather model trained successfully!
Space weather at Earth orbit: {'solar_flare_risk': 'High', 'radiation_level': 'Moderate'}
Space weather at Moon surface: {'solar_flare_risk': 'High', 'radiation_level': 'Moderate'}
Available trajectories: [{'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}]




OODA Loop:  20%|██        | 1/5 [02:04<08:16, 124.05s/it]




Generated Space Flight Plan:

            <s>[INST] Generate a detailed space flight plan for a mission from Earth orbit to the Moon surface,
            using NASA's Lunar Gateway as a staging point. Consider the following:
            - Chosen Trajectory: {'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}
            - Solar flare risk at origin: High
            - Solar flare risk at destination: High
            # ... (Add other instructions/details to the prompt) ...
            [/INST] 8
            def generate_space_flight_plan():
    # Define the mission parameters
    trajectory_id = 'T1'
    delta_v = 3.9
    duration = 3.5
    fuel_required = 1200
    waypoints = ['Lunar Gateway']
    solar_flare_risk_origin = 'High'
    solar_flare_risk_destination = 'High'
    # Define the space flight plan
    plan = {
        'trajectory_id': trajectory_id,
        'delta_v': delta_v,
        'duration': duration,
        '

OODA Loop:  40%|████      | 2/5 [02:32<03:23, 67.78s/it] 




Generated Space Flight Plan:

            <s>[INST] Generate a detailed space flight plan for a mission from Earth orbit to the Moon surface,
            using NASA's Lunar Gateway as a staging point. Consider the following:
            - Chosen Trajectory: {'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}
            - Solar flare risk at origin: High
            - Solar flare risk at destination: High
            # ... (Add other instructions/details to the prompt) ...
            [/INST]
```python
import math
def calculate_flight_plan(trajectory_id, delta_v, duration, fuel_required, waypoints):
    """
    Calculate the flight plan for a mission from Earth orbit to the Moon surface,
    using NASA's Lunar Gateway as a staging point.
    Args:
        trajectory_id (str): Unique identifier for the chosen trajectory.
        delta_v (float): Change in velocity required for the mission.
        duration (float): Duration 

...




Generated Space Flight Plan:

            <s>[INST] Generate a detailed space flight plan for a mission from Earth orbit to the Moon surface,
            using NASA's Lunar Gateway as a staging point. Consider the following:
            - Chosen Trajectory: {'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}
            - Solar flare risk at origin: High
            - Solar flare risk at destination: High
            # ... (Add other instructions/details to the prompt) ...
            [/INST]           
**Mission Briefing**
The objective of this mission is to transport a crew from Earth orbit to the Moon's surface, utilizing the Lunar Gateway as a critical staging point. The chosen trajectory, denoted as T1, has been selected based on its optimal energy efficiency and safety considerations.
**Mission Parameters**
- **Trajectory ID:** T1
- **Delta-V:** 3.9 km/s
- **Duration:** 3.5 days
- **Fuel Required:** 1200 kg
- **Waypoi

OODA Loop:  80%|████████  | 4/5 [05:11<01:10, 70.38s/it]




Generated Space Flight Plan:

            <s>[INST] Generate a detailed space flight plan for a mission from Earth orbit to the Moon surface,
            using NASA's Lunar Gateway as a staging point. Consider the following:
            - Chosen Trajectory: {'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}
            - Solar flare risk at origin: High
            - Solar flare risk at destination: High
            # ... (Add other instructions/details to the prompt) ...
            [/INST]s
**Mission Overview**
The proposed mission, named "Lunar Pioneer," aims to transport a crew from Earth orbit to the Moon's surface using NASA's Lunar Gateway as a staging point. The chosen trajectory, T1, involves a series of complex maneuvers to ensure a safe and efficient journey.
**Trajectory Details**
---------------------
#

**Trajectory ID**:
*   `T1`
#

**Delta-V (Δv)**:
*   3.9 km/s
#

**Duration**:
*   3.5 days
#

**Fuel Requi

OODA Loop: 100%|██████████| 5/5 [07:14<00:00, 86.95s/it]




Generated Space Flight Plan:

            <s>[INST] Generate a detailed space flight plan for a mission from Earth orbit to the Moon surface,
            using NASA's Lunar Gateway as a staging point. Consider the following:
            - Chosen Trajectory: {'trajectory_id': 'T1', 'delta_v': 3.9, 'duration': 3.5, 'fuel_required': 1200, 'waypoints': ['Lunar Gateway']}
            - Solar flare risk at origin: High
            - Solar flare risk at destination: High
            # ... (Add other instructions/details to the prompt) ...
            [/INST]           
**Mission Overview**
The proposed space flight plan aims to transport a crew from Earth orbit to the Moon surface, utilizing the NASA's Lunar Gateway as a staging point. The chosen trajectory, denoted as T1, involves a series of maneuvers to ensure a safe and efficient journey.
**Trajectory Details**
| Trajectory ID | ΔV (m/s) | Duration (days) | Fuel Required (kg) | Waypoints |
| --- | --- | --- | --- | --- |
| T1 | 3.9 | 3